<a href="https://colab.research.google.com/github/KadinRelefourd/neuralnetworkclassifier/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab inviornment

Imports

In [2]:
import os
import cv2  # OpenCV for image processing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

Prepare Training and Testing data

In [3]:
transform = transforms.ToTensor()

train_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=True,
    transform=transform,
    download=True
)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=False,
    transform=transform,
    download=True
)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:18<00:00, 9.10MB/s]


Extracting ./data.cifar10/cifar-10-python.tar.gz to ./data.cifar10
Files already downloaded and verified


Build model

In [21]:
class My_XXXnet(nn.Module):
    def __init__(self):
        super(My_XXXnet, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)  # Input size: CIFAR-10 images are 32x32x3
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)  # Output size: 10 classes (CIFAR-10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input image
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)  # Apply dropout for regularization
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc3(x))
        x = F.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = My_XXXnet()
loss_func = nn.CrossEntropyLoss()  # CrossEntropyLoss includes softmax internally
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

Testing Function

In [12]:
def test():
    model.eval()  # Switch to evaluation mode
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    return total_loss / len(test_loader), accuracy

Training Accuracy

In [13]:
def train_accuracy():
    model.eval()  # Switch to evaluation mode for accuracy calculation on training data
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return 100 * correct / total

Save Model function

In [18]:
def save_model(epoch):
    if not os.path.exists('./model'):
        os.makedirs('./model')
    torch.save(model.state_dict(), './model/model.ckpt')  # Save with fixed filename
    print(f"Model saved in file: ./model/model.ckpt")

Load Model Funciont

In [7]:
def load_model(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Model loaded successfully.")

Train Function


In [17]:
def train():
    print(f"{'Loop':<8}{'Train Loss':<15}{'Train Acc %':<15}{'Test Loss':<15}{'Test Acc %':<15}")

    last_train_acc = 0  # Track the last training accuracy

    for epoch in range(10):  # Train for 10 epochs
        model.train()  # Set the model to training mode

        total_loss = 0.0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Evaluate after each epoch
        train_acc = train_accuracy()
        test_loss, test_accuracy = test()

        # Ensure training accuracy only goes up
        if train_acc < last_train_acc:
            print(f"Warning: Training accuracy decreased from {last_train_acc:.2f}% to {train_acc:.2f}%")
        last_train_acc = train_acc  # Update last training accuracy

        print(f"{epoch + 1}/10   {total_loss / len(train_loader):<15.4f}{train_acc:<15.2f}{test_loss:<15.4f}{test_accuracy:<15.2f}")

        # Save the model only after the last epoch (10th epoch)
        if epoch == 9:
            save_model(epoch)


Test Function

In [9]:
def classify(image_path):
    load_model('./model/model_epoch_9.pt')  # Load the best model from training
    model.eval()

    # Read the image using OpenCV and preprocess it
    img = cv2.imread(image_path)  # Load image as a NumPy array (BGR format)

    if img is None:
        print("Error: Image not found or invalid path.")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB format (CIFAR-10 uses RGB)
    img_resized = cv2.resize(img, (32, 32))  # Resize to CIFAR-10 dimensions (32x32)

    img_tensor = transform(img_resized).unsqueeze(0)  # Convert to tensor and add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_class = torch.max(output.data, 1)

    classes = train_data.classes
    print(f'Prediction result: {classes[predicted_class.item()]}')

Command Line prompts

In [10]:
if __name__ == "__main__":
    import sys

    command = sys.argv[1]

    if command == "train":
        train()

    elif command == "test":
        image_path = sys.argv[2]
        classify(image_path)

Run Training

In [ ]:
train()

Loop    Train Loss     Train Acc %    Test Loss      Test Acc %     
1/10   2.1182         22.18          2.0394         21.92          
2/10   2.0045         25.01          1.9670         25.00          
